# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from sympy import *
from sympy import solve

from PIL import Image

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))
    end_points = np.zeros((6, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break
        # if you have drawn 3 lines, force to stop
        if n == 3:
            break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        
        one_end_points = np.concatenate((pt1, pt2), axis=None)
        end_points = np.append(end_points, one_end_points.reshape((6, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers, end_points

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
# get this post from wiki for line intercept
# https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection

def findIntersection(x1, y1, x2, y2, x3, y3, x4, y4):
    px= ( (x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) ) 
    py= ( (x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4) ) / ( (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4) )
    return np.array([px, py, 1])

def get_vanishing_point(end_points):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    p1 = end_points[0:2, 0]
    p2 = end_points[3:5, 0]
    p3 = end_points[0:2, 1]
    p4 = end_points[3:5, 1]
    p5 = end_points[0:2, 2]
    p6 = end_points[3:5, 2]
    
    vp1 = findIntersection(p1[0], p1[1], p2[0], p2[1], p3[0], p3[1], p4[0], p4[1])
    vp2 = findIntersection(p5[0], p5[1], p6[0], p6[1], p3[0], p3[1], p4[0], p4[1])
    vp = (vp1 + vp2) / 2
    
    print('coord of vanishing point: ')
    print(vp)
    return vp

In [6]:
# using cross to connect the vanishing points togethter

def get_horizon_line(vanishing_points):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    # we can only use the two finite vanishing points
    p1 = vanishing_points[:, 0]
    p2 = vanishing_points[:, 1]
    
    # normalize it to a^2 + b^2
    horizon = np.cross(p1, p2)
    horizon = horizon / np.sqrt(horizon[0]**2 + horizon[1]**2)
    
    print('get horizon param: ')
    print(horizon)
    return horizon, vanishing_points[:, 0:2]

In [7]:
# plot the horizon line with two finite vanishing points

def plot_horizon_line(im, vp):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    x1 = vp[0][0]
    x2 = vp[0][1]
    y1 = vp[1][0]
    y2 = vp[1][1]
    
    slope = (y2 - y1)/(x2 - x1)
    intercept = y1 - slope * x1
    
    right_end = slope * im.shape[1] + intercept
    
    plt.figure()
    plt.imshow(im)
    plt.plot([0, im.shape[1]], [intercept, right_end], color='b')
    plt.show()

In [8]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    # this is the intrinsic matrix of camera by v_i^TK^(-T)K^(-1)v_j = 0
    vpt1 = vpts[:, 0][:, np.newaxis]
    vpt2 = vpts[:, 1][:, np.newaxis]
    vpt3 = vpts[:, 2][:, np.newaxis]
    
    f, px, py = symbols('f, px, py')
    Kinv = Matrix([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]])
    
    # three orthogonal equation
    m1 = vpt1.T * Kinv.T * Kinv * vpt2
    m2 = vpt1.T * Kinv.T * Kinv * vpt3
    m3 = vpt2.T * Kinv.T * Kinv * vpt3
    f, px, py = solve([m1[0], m2[0], m3[0]], (f, px, py))[0]
    
    return abs(f), px, py

In [9]:
def get_rotation_matrix(vpts, f, px, py):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # r_i = lambda_i * K^(-1) * v_i
    vpt1 = vpts[:, 0][:, np.newaxis]
    vpt2 = vpts[:, 1][:, np.newaxis]
    vpt3 = vpts[:, 2][:, np.newaxis]
    
    Kinv = np.array([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]]).astype(np.float)
    r1 = Kinv.dot(vpt1)
    r2 = Kinv.dot(vpt2)
    r3 = Kinv.dot(vpt3)
    
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    return np.concatenate((r1, r2, r3), axis=1)

In [10]:
# matching all coords to horizon line to calculate its height
def estimate_height(im, obj, coords, horizon, standard_height, finite_vanishing_points):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    infinite_vp = vpts[:, 2]
    
    person_top = coords['person'][:, 0]
    person_bottom = coords['person'][:, 1]

    obj_top = coords[obj][:, 0]
    obj_bottom = coords[obj][:, 1]

    two_bottom_connected = np.cross(person_bottom, obj_bottom)
    bottom_horizon_intersect = np.cross(two_bottom_connected, horizon)
    bottom_horizon_intersect = bottom_horizon_intersect / bottom_horizon_intersect[-1]
    
    persontop_object = np.cross(bottom_horizon_intersect, person_top)
    estimated_obj_line = np.cross(obj_top, obj_bottom)
    object_got_intercept = np.cross(persontop_object, estimated_obj_line)
    object_got_intercept = object_got_intercept / object_got_intercept[-1]
    
    invariant1 = np.linalg.norm(obj_top - obj_bottom)
    invariant2 = np.linalg.norm(infinite_vp - object_got_intercept)
    invariant3 = np.linalg.norm(object_got_intercept - obj_bottom)
    invariant4 = np.linalg.norm(infinite_vp - obj_top)
    
    height = standard_height * (invariant1 * invariant2 / invariant3 / invariant4)
    
    # all the invariant lines we calculated above
    plt.figure()
    plt.imshow(im)
    # plot the standard person line
    plt.plot([person_top[0], person_bottom[0]], [person_top[1], person_bottom[1]])
    # plot the obj
    plt.plot([obj_top[0], obj_bottom[0]], [obj_top[1], obj_bottom[1]])
    # plot the line connecting the bottoms of obj and reference person
    plt.plot([person_bottom[0], obj_bottom[0]], [person_bottom[1], obj_bottom[1]])
    # plot the line connecting the top of ...
    plt.plot([person_top[0], obj_top[0]], [person_top[1], obj_top[1]])
    # plot the vanishing lines
    def plot_horizon_line_inside(im, vp):
        x1 = vp[0][0]
        x2 = vp[0][1]
        y1 = vp[1][0]
        y2 = vp[1][1]
        slope = (y2 - y1)/(x2 - x1)
        intercept = y1 - slope * x1
        right_end = slope * im.shape[1] + intercept
        plt.plot([0, im.shape[1]], [intercept, right_end], color='b')

    plot_horizon_line_inside(im, finite_vanishing_points)
    
    plt.show()
    
    return height

# Main function

In [11]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers, end_points = get_input_lines(im)
    
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    print("end of selecting lines")
    
    vpts[:, i] = get_vanishing_point(end_points)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon, finite_vanishing_points = get_horizon_line(vpts)

# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, finite_vanishing_points)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
end of selecting lines
coord of vanishing point: 
[-746.25322053  252.86976347    1.        ]
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
end of selecting lines
coord of vanishing poin

/Users/wangluning/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/Users/wangluning/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
/Users/wangluning/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/Users/wangluning/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [29]:
# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
print('focal length: '+ str(f))
print('optical center x: ' + str(u))
print('optical center y: ' + str(v))

focal length: 789.080216878090
optical center x: 480.784050745996
optical center y: 293.382064049728


In [34]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, u, v)
print('rotation matrix: ' + str(R))

rotation matrix: [[-0.69196587  0.72187136  0.00921808]
 [-0.07092302 -0.08068065  0.99421354]
 [ 0.718438    0.68730806  0.10702556]]


In [36]:
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
standard_height = 1.6764  # of the person next to the spike

for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, obj, coords, horizon, standard_height, finite_vanishing_points)
    print("Estimated height of {} is {} m".format(obj, height))


Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
Estimated height of CSL building is 38.16283119433413 m
Estimating height of the spike statue
Estimated height of the spike statue is 9.796303648082798 m
Estimating height of the lamp posts
Estimated height of the lamp posts is 4.536315680271095 m


/Users/wangluning/miniconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [37]:
# <YOUR IMPLEMENTATION> Estimate heights
standard_height = 1.8288 # of the person next to the spike

for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, obj, coords, horizon, standard_height, finite_vanishing_points)
    print("Estimated height of {} is {} m".format(obj, height))

Estimating height of CSL building
Estimated height of CSL building is 41.63217948472814 m
Estimating height of the spike statue
Estimated height of the spike statue is 10.686876706999417 m
Estimating height of the lamp posts
Estimated height of the lamp posts is 4.948708014841195 m


# Extra Credit

In [41]:
extra_objects = ('person', 'person1', 'person2', 'person3', 'window')
coords = dict()
for obj in extra_objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
standard_height = 1.6764  # of the person next to the spike

for obj in extra_objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, obj, coords, horizon, standard_height, finite_vanishing_points)
    print("Estimated height of {} is {} m".format(obj, height))


Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of person1
Click on the bottom coordinate of person1
Click on the top coordinate of person2
Click on the bottom coordinate of person2
Click on the top coordinate of person3
Click on the bottom coordinate of person3
Click on the top coordinate of window
Click on the bottom coordinate of window
Estimating height of person1
Estimated height of person1 is 1.4964519010597526 m
Estimating height of person2
Estimated height of person2 is 1.625622621491757 m
Estimating height of person3
Estimated height of person3 is 1.8235501563297507 m
Estimating height of window
Estimated height of window is 3.4546059910628157 m
